In [1]:
# main.py

from vector import Vector
from body import Body
from renderer import PygameRenderer

# Define our constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GRAVITY = Vector(0, 9.8)
DELTA_TIME = 0.01

# --- This is the function we've filled in! ---
def check_for_collision(body1, body2, radius):
    # 1. Calculate the vector from body1 to body2 using our Vector subtraction
    direction_vector = body2.position - body1.position
    
    # 2. Calculate the squared magnitude (distance squared) of that vector
    #    This is a super fast and efficient way to do it.
    distance_squared = direction_vector.x**2 + direction_vector.y**2
    
    # 3. Calculate the squared sum of the radii
    radii_sum = radius + radius
    radii_sum_squared = radii_sum**2
    
    # 4. Check if a collision has occurred!
    return distance_squared <= radii_sum_squared

# --- Setup ---
renderer = PygameRenderer(SCREEN_WIDTH, SCREEN_HEIGHT)

# Create two body objects with different colors
body1 = Body(mass=10.0, position=Vector(SCREEN_WIDTH / 4, 50), velocity=Vector(70, 0), color=(255, 0, 0)) # Red
body2 = Body(mass=10.0, position=Vector(SCREEN_WIDTH * 3 / 4, 50), velocity=Vector(-70, 0), color=(0, 0, 255)) # Blue
bodies = [body1, body2]
BODY_RADIUS = 10

# --- Main Simulation Loop ---
running = True
while running:
    # Handle user input and events (like closing the window)
    renderer.check_for_quit()

    # --- Physics update step ---
    for body in bodies:
        body.update(DELTA_TIME, GRAVITY)
        body.check_and_resolve_collisions(SCREEN_WIDTH, SCREEN_HEIGHT, BODY_RADIUS)

    # --- Check for collisions between bodies ---
    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            # We are checking all pairs of bodies
            if check_for_collision(bodies[i], bodies[j], BODY_RADIUS):
                # Now we know they've collided!
                print("Collision Detected!")


    # --- Rendering step ---
    # 1. Clear the screen
    renderer.clear_screen()
    
    # 2. Draw all objects
    for body in bodies:
        renderer.draw_body(body, color=body.color, radius=BODY_RADIUS)

    # 3. Update the display to show what we've drawn
    renderer.flip_display()

The sum of Vector(3.0, 5.0) and Vector(1.0, -2.0) is Vector(4.0, 3.0)
The difference of Vector(3.0, 5.0) and Vector(1.0, -2.0) is Vector(2.0, 7.0)
The vector Vector(3.0, 5.0) scaled by 3 is Vector(9.0, 15.0)
The vector Vector(1.0, -2.0) scaled by 2 is Vector(2.0, -4.0)
pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!
Collision Detected!


SystemExit: 

d:\Programming\Python\Engine v1\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# main.py

from vector import Vector
from body import Body
from renderer import PygameRenderer
import math # We need this for math.sqrt

# Define our constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GRAVITY = Vector(0, -9.8)
DELTA_TIME = 0.01
COEFFICIENT_OF_RESTITUTION = 0.8 # 1.0 is perfectly elastic, 0.0 is perfectly inelastic

# --- Collision Detection ---
def check_for_collision(body1, body2, radius):
    direction_vector = body2.position - body1.position
    distance_squared = direction_vector.x**2 + direction_vector.y**2
    radii_sum = radius + radius
    radii_sum_squared = radii_sum**2
    return distance_squared <= radii_sum_squared

# --- Collision Resolution ---
def resolve_collision(body1, body2, radius):
    # Step 1: Separate the bodies if they are overlapping
    direction_vector = body2.position - body1.position
    distance = direction_vector.magnitude()
    overlap = (2 * radius) - distance
    if distance != 0:
        separation_vector = direction_vector.normalize() * (overlap / 2)
        body1.position -= separation_vector
        body2.position += separation_vector
    
    # Step 2: Calculate the normal vector (line of impact)
    collision_normal = (body2.position - body1.position).normalize()
    
    # Step 3: Calculate the relative velocity
    relative_velocity = body2.velocity - body1.velocity
    
    # Step 4: Calculate the relative velocity along the normal
    # This is a dot product, which we don't have yet, so we'll do it manually.
    normal_velocity = relative_velocity.x * collision_normal.x + relative_velocity.y * collision_normal.y
    
    # If bodies are moving away from each other, don't apply an impulse
    if normal_velocity > 0:
        return
    
    # Step 5: Calculate the impulse magnitude, with the coefficient of restitution
    j = (-(1.0 + COEFFICIENT_OF_RESTITUTION) * normal_velocity) / (1 / body1.mass + 1 / body2.mass)
    
    # Step 6: Apply the impulse to the velocities
    impulse = collision_normal * j
    body1.velocity -= impulse / body1.mass
    body2.velocity += impulse / body2.mass
    
# --- Setup ---
renderer = PygameRenderer(SCREEN_WIDTH, SCREEN_HEIGHT)

body1 = Body(mass=10.0, position=Vector(SCREEN_WIDTH / 4, 50), velocity=Vector(70, 1500), color=(255, 0, 0))
body2 = Body(mass=10.0, position=Vector(SCREEN_WIDTH * 3 / 4, 50), velocity=Vector(0, 0), color=(0, 0, 255))
bodies = [body1, body2]
BODY_RADIUS = 10

# --- Main Simulation Loop ---
running = True
while running:
    renderer.check_for_quit()

    for body in bodies:
        body.update(DELTA_TIME, GRAVITY)
        # --- This is the updated line! ---
        body.check_and_resolve_collisions(SCREEN_WIDTH, SCREEN_HEIGHT, BODY_RADIUS, COEFFICIENT_OF_RESTITUTION)

    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            if check_for_collision(bodies[i], bodies[j], BODY_RADIUS):
                resolve_collision(bodies[i], bodies[j], BODY_RADIUS)

    renderer.clear_screen()
    for body in bodies:
        renderer.draw_body(body, color=body.color, radius=BODY_RADIUS)
    renderer.flip_display()

SystemExit: 

In [1]:
# main.py

from vector import Vector
from body import Body
from renderer import PygameRenderer
import math

# Define our constants
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
GRAVITY = Vector(0, 9.8)
DELTA_TIME = 0.01
COEFFICIENT_OF_RESTITUTION = 0.8 # 1.0 is perfectly elastic, 0.0 is perfectly inelastic
DRAG_COEFFICIENT = 0.05 # A small value for air resistance

# --- Collision Detection ---
def check_for_collision(body1, body2, radius):
    direction_vector = body2.position - body1.position
    distance_squared = direction_vector.x**2 + direction_vector.y**2
    radii_sum = radius + radius
    radii_sum_squared = radii_sum**2
    return distance_squared <= radii_sum_squared

# --- Collision Resolution ---
def resolve_collision(body1, body2, radius):
    # Step 1: Separate the bodies if they are overlapping
    direction_vector = body2.position - body1.position
    distance = direction_vector.magnitude()
    overlap = (2 * radius) - distance
    if distance != 0:
        separation_vector = direction_vector.normalize() * (overlap / 2)
        body1.position -= separation_vector
        body2.position += separation_vector
    
    # Step 2: Calculate the normal vector (line of impact)
    collision_normal = (body2.position - body1.position).normalize()
    
    # Step 3: Calculate the relative velocity
    relative_velocity = body2.velocity - body1.velocity
    
    # Step 4: Calculate the relative velocity along the normal
    # This is a dot product, which we don't have yet, so we'll do it manually.
    normal_velocity = relative_velocity.x * collision_normal.x + relative_velocity.y * collision_normal.y
    
    # If bodies are moving away from each other, don't apply an impulse
    if normal_velocity > 0:
        return
    
    # Step 5: Calculate the impulse magnitude, with the coefficient of restitution
    j = (-(1.0 + COEFFICIENT_OF_RESTITUTION) * normal_velocity) / (1 / body1.mass + 1 / body2.mass)
    
    # Step 6: Apply the impulse to the velocities
    impulse = collision_normal * j
    body1.velocity -= impulse / body1.mass
    body2.velocity += impulse / body2.mass
    
# --- Setup ---
renderer = PygameRenderer(SCREEN_WIDTH, SCREEN_HEIGHT)

body1 = Body(mass=10.0, position=Vector(SCREEN_WIDTH / 4, 50), velocity=Vector(70, 0), color=(255, 0, 0))
body2 = Body(mass=10.0, position=Vector(SCREEN_WIDTH * 3 / 4, 50), velocity=Vector(-70, 0), color=(0, 0, 255))
bodies = [body1, body2]
BODY_RADIUS = 10

# --- Main Simulation Loop ---
running = True
while running:
    renderer.check_for_quit()

    for body in bodies:
        body.update(DELTA_TIME, GRAVITY, DRAG_COEFFICIENT)
        body.check_and_resolve_collisions(SCREEN_WIDTH, SCREEN_HEIGHT, BODY_RADIUS, COEFFICIENT_OF_RESTITUTION)

    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            if check_for_collision(bodies[i], bodies[j], BODY_RADIUS):
                resolve_collision(bodies[i], bodies[j], BODY_RADIUS)

    renderer.clear_screen()
    for body in bodies:
        renderer.draw_body(body, color=body.color, radius=BODY_RADIUS)
    renderer.flip_display()

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

d:\Programming\Python\Engine v1\venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import pygame
import math
import sys

# ... [Your Vector and Body classes here] ...
class Vector:
    """
    A simple 2D Vector class.
    """
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

    def __truediv__(self, scalar):
        return Vector(self.x / scalar, self.y / scalar)
    
    def magnitude(self):
        """Calculates the magnitude of the vector."""
        return math.sqrt(self.x**2 + self.y**2)

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

class Body:
    """
    A simple physical object with mass, position, and velocity.
    """
    def __init__(self, mass=1.0, position=Vector(0, 0), velocity=Vector(0, 0), radius=10, color=(255, 255, 255)):
        self.mass = float(mass)
        self.position = position
        self.velocity = velocity
        self.force = Vector(0, 0)
        self.radius = radius
        self.color = color
        self.is_dragged = False

    def apply_force(self, force_vector):
        self.force += force_vector

    def update(self, dt):
        if not self.is_dragged:
            acceleration = self.force / self.mass
            self.velocity += acceleration * dt
            self.position += self.velocity * dt

        self.force = Vector(0, 0)

    def draw(self, screen):
        """Draws the body as a circle on the Pygame screen."""
        pygame.draw.circle(screen, self.color, (int(self.position.x), int(self.position.y)), self.radius)

    def __repr__(self):
        return f"Body(mass={self.mass}, pos={self.position}, vel={self.velocity})"

# --- Pygame Initialization ---
pygame.init()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Physics Simulation")

clock = pygame.time.Clock()

# --- Simulation Setup ---
GRAVITY = Vector(0, 0.5)  # Let's use a smaller gravity for now
bodies = [
    Body(mass=10.0, position=Vector(100, 100), velocity=Vector(2, 0), radius=15, color=(255, 0, 0)),
    Body(mass=5.0, position=Vector(400, 200), velocity=Vector(0, 3), radius=10, color=(0, 255, 0))
]

dragged_body = None

# --- Main Game Loop ---
running = True
while running:
    dt = clock.tick(60) / 1000.0  # Time step in seconds

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1: # Left mouse button
                mouse_pos = Vector(event.pos[0], event.pos[1])
                for body in bodies:
                    # Check if mouse is over the body
                    if (body.position - mouse_pos).magnitude() <= body.radius:
                        dragged_body = body
                        dragged_body.is_dragged = True
                        dragged_body.velocity = Vector(0, 0) # Stop its motion
                        break # Found a body, so stop checking
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                if dragged_body:
                    dragged_body.is_dragged = False
                    dragged_body = None
        elif event.type == pygame.MOUSEMOTION:
            if dragged_body:
                mouse_pos = Vector(event.pos[0], event.pos[1])
                dragged_body.position = mouse_pos
                # We'll calculate a new velocity on mouse up later for a better feel.

    # Physics updates
    for body in bodies:
        if not body.is_dragged:
            body.apply_force(GRAVITY * body.mass)
        body.update(dt)

    # Drawing
    screen.fill((0, 0, 0))  # Clear the screen
    for body in bodies:
        body.draw(screen)

    pygame.display.flip()  # Update the display

pygame.quit()
sys.exit()

SystemExit: 

In [3]:
import pygame
import math
import sys

class Vector:
    """
    A simple 2D Vector class.
    """
    def __init__(self, x=0.0, y=0.0):
        self.x = float(x)
        self.y = float(y)

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        return Vector(self.x * scalar, self.y * scalar)

    def __rmul__(self, scalar):
        return self.__mul__(scalar)

    def __truediv__(self, scalar):
        return Vector(self.x / scalar, self.y / scalar)
    
    def magnitude_squared(self):
        """Calculates the squared magnitude (length) of the vector. More efficient than magnitude()."""
        return self.x**2 + self.y**2

    def magnitude(self):
        """Calculates the magnitude of the vector."""
        return math.sqrt(self.magnitude_squared())
    
    def normalize(self):
        """Returns a new vector with the same direction but a magnitude of 1."""
        mag = self.magnitude()
        if mag == 0:
            return Vector(0, 0)
        return self / mag

    def dot(self, other):
        """Calculates the dot product of two vectors."""
        return self.x * other.x + self.y * other.y

    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

class Body:
    """
    A simple physical object with mass, position, and velocity.
    """
    def __init__(self, mass=1.0, position=Vector(0, 0), velocity=Vector(0, 0), radius=10, color=(255, 255, 255)):
        self.mass = float(mass)
        self.position = position
        self.velocity = velocity
        self.force = Vector(0, 0)
        self.radius = radius
        self.color = color
        self.is_dragged = False
        self.original_color = color

    def apply_force(self, force_vector):
        self.force += force_vector

    def update(self, dt):
        if not self.is_dragged:
            acceleration = self.force / self.mass
            self.velocity += acceleration * dt
            self.position += self.velocity * dt

        self.force = Vector(0, 0)

    def draw(self, screen):
        """Draws the body as a circle on the Pygame screen."""
        pygame.draw.circle(screen, self.color, (int(self.position.x), int(self.position.y)), self.radius)

# --- Collision Resolution Function ---
def resolve_collision(body1, body2):
    """
    Calculates the new velocities for two bodies after an elastic collision.
    """
    # Vector from body2 to body1
    n = body1.position - body2.position
    
    # Check for overlapping bodies and separate them
    distance = n.magnitude()
    if distance == 0:
        return # Avoid division by zero
    
    # Collision normal and tangent vectors
    normal = n.normalize()
    
    # Relative velocity
    v_rel = body1.velocity - body2.velocity
    
    # The dot product of v_rel and the normal tells us if the bodies are moving away from each other
    if v_rel.dot(normal) >= 0:
        return # Already moving apart, no need to resolve collision
    
    # Calculate impulse scalar (change in momentum)
    j = -2 * v_rel.dot(normal) / (1 / body1.mass + 1 / body2.mass)
    
    # Apply impulse to velocities
    body1.velocity += normal * (j / body1.mass)
    body2.velocity -= normal * (j / body2.mass)
    
    # Optional: Separate bodies to prevent them from getting stuck
    overlap = (body1.radius + body2.radius) - distance
    body1.position += normal * (overlap / 2)
    body2.position -= normal * (overlap / 2)


# --- Pygame Initialization and Main Loop ---
pygame.init()

SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Physics Simulation")

clock = pygame.time.Clock()

# --- Simulation Setup ---
GRAVITY = Vector(0, 0.5)
bodies = [
    Body(mass=10.0, position=Vector(100, 100), velocity=Vector(20, 0), radius=20, color=(255, 0, 0)),
    Body(mass=5.0, position=Vector(400, 100), velocity=Vector(-10, 0), radius=15, color=(0, 255, 0)),
    Body(mass=8.0, position=Vector(250, 400), velocity=Vector(5, -10), radius=18, color=(0, 0, 255))
]

dragged_body = None
mouse_pos_history = []

# --- Main Game Loop ---
running = True
while running:
    dt = clock.tick(60) / 1000.0

    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:
                mouse_pos = Vector(event.pos[0], event.pos[1])
                for body in bodies:
                    if (body.position - mouse_pos).magnitude() <= body.radius:
                        dragged_body = body
                        dragged_body.is_dragged = True
                        dragged_body.velocity = Vector(0, 0)
                        break
        elif event.type == pygame.MOUSEBUTTONUP:
            if event.button == 1:
                if dragged_body:
                    dragged_body.is_dragged = False
                    # Give the released body a velocity based on the last mouse movements
                    if len(mouse_pos_history) > 1:
                        last_pos = mouse_pos_history[-2]
                        current_pos = mouse_pos_history[-1]
                        dragged_body.velocity = (current_pos - last_pos) * 10
                    dragged_body = None
                    mouse_pos_history = []
        elif event.type == pygame.MOUSEMOTION:
            if dragged_body:
                mouse_pos = Vector(event.pos[0], event.pos[1])
                dragged_body.position = mouse_pos
                mouse_pos_history.append(mouse_pos)
                if len(mouse_pos_history) > 5: # Keep a short history
                    mouse_pos_history.pop(0)

    # Physics and Collision Updates
    for body in bodies:
        if not body.is_dragged:
            body.apply_force(GRAVITY * body.mass)
        body.update(dt)

    # Collision Detection and Response (with a nested loop to check all pairs)
    for i in range(len(bodies)):
        for j in range(i + 1, len(bodies)):
            body1 = bodies[i]
            body2 = bodies[j]
            # Check for collision
            dist_sq = (body1.position - body2.position).magnitude_squared()
            radius_sum = body1.radius + body2.radius
            if dist_sq <= radius_sum**2:
                if not body1.is_dragged and not body2.is_dragged:
                    # Change color to visualize collision
                    body1.color = (255, 255, 255)
                    body2.color = (255, 255, 255)
                    resolve_collision(body1, body2)
                
    # Reset colors after collision is resolved
    for body in bodies:
        body.color = body.original_color

    # Drawing
    screen.fill((0, 0, 0))
    for body in bodies:
        body.draw(screen)

    pygame.display.flip()

pygame.quit()
sys.exit()

SystemExit: 